In [1]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd

In [2]:
# Valori pro capite
dataset = '93_1227_DF_DCCN_TNA1_6'
data_url = f'https://esploradati.istat.it/SDMXWS/rest/data/IT1,{dataset},1.0/A........./ALL/?detail=full&startPeriod=2014-01-01&endPeriod=2023-12-31&dimensionAtObservation=TIME_PERIOD'
response = requests.get(data_url)
root = ET.fromstring(response.content)
# 🔎 Namespace XML per SDMX aggiornato
ns = {"message": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/message","generic": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/data/generic"}
data = []
for series in root.findall(".//generic:Series", namespaces=ns):
    series_key = series.find("generic:SeriesKey", namespaces=ns)
    attributes = {value.attrib["id"]: value.attrib["value"] for value in series_key.findall("generic:Value", namespaces=ns)}
    for obs in series.findall("generic:Obs", namespaces=ns):
        time_period = obs.find("generic:ObsDimension", namespaces=ns).attrib["value"]
        obs_value = obs.find("generic:ObsValue", namespaces=ns).attrib["value"]
        data.append({**attributes, "TIME_PERIOD": time_period, "OBS_VALUE": float(obs_value)})
df = pd.DataFrame(data)
# Valori pro capite
df1 = df.query('VALUATION == "V" and TIME_PERIOD == "2022" and DATA_TYPE_AGGR in ["B1G_B_W2_S1_R_POP","B1GQ_B_W2_S1_R_POP","B1G_B_W2_S1"]'
).replace("B1GQ_B_W2_S1_R_POP", "PIL/abitante").replace("B1G_B_W2_S1_R_POP", "Val.Aggiunto/abitante")[['EDITION','TIME_PERIOD', 'REF_AREA', 'DATA_TYPE_AGGR', 'OBS_VALUE']]
ValProCapite = pd.pivot_table(df1, index = 'REF_AREA', columns='DATA_TYPE_AGGR', values = 'OBS_VALUE')


In [3]:
ValProCapite

DATA_TYPE_AGGR,PIL/abitante,Val.Aggiunto/abitante
REF_AREA,,
IT,33849.1525,30384.3890
IT108,35902.5230,32196.6740
IT109,27444.0810,24599.9405
IT110,18284.2670,16411.6020
IT111,17877.2715,16044.9805
...,...,...
ITG25,24843.3885,22298.7175
ITG26,22217.6530,19942.8285
ITG27,32173.3965,28878.8600


In [ ]:
ValProCapite.to_excel(writer, sheet_name='ValProCapite', startrow=3, startcol=2)

sheet_to_update = tabella
book = load_workbook(file_path)
with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df.to_excel(writer, sheet_name=sheet_to_update, index=False)